In [1]:
import pandas as pd

In [2]:
#----------------------------------------------------------------------------------------
#
#
# 1. Mapping Fixations and Saccades to AOIs
#
#
#----------------------------------------------------------------------------------------

In [3]:
# A list containing the AOIs considered in the study --- **TO UPDATE IF YOU UPDATE THE AOIs**
AOIS = ['Paragraph1', 'Paragraph2', 'Paragraph3', 'Paragraph4',
       'Paragraph5', 'Paragraph6', 'Text_Area', 'SubFigure1', 'SubFigure2',
       'SubFigure3', 'Figure', 'Formula']

In [4]:
# Read gaze data with AOIs and events using pandas library
data = pd.read_csv("data/gazeDataWithAOIsAndEvents.csv")
# set display.max_columns to none, to show all the columns when using head()
pd.set_option('display.max_columns', None)

In [5]:
# Preview data
data.head()

,Row,Respondent,Age,Gender,Group,Calibration,Timestamp,SourceStimuliName,EventSource.1,ET_TimeSignal,ET_CameraLeftX,ET_CameraLeftY,ET_CameraRightX,ET_CameraRightY,ET_ValidityLeft,ET_ValidityRight,GazeX,GazeY,Paragraph1,Paragraph2,Paragraph3,Paragraph4,Paragraph5,Paragraph6,Text_Area,SubFigure1,SubFigure2,SubFigure3,Figure,Formula,InterpolatedGazeX,InterpolatedGazeY,InterpolatedDistance,GazeVelocityAngle,FixID,Fixation Index by Stimulus,Fixation X,Fixation Y,Fixation Start,Fixation End,Fixation Duration,Fixation Dispersion,ET_PupilLeft,GazeAccelerationAngle,ET_PupilRight,SacID,Saccade Index by Stimulus,Saccade Start,Saccade End,Saccade Duration,Saccade Amplitude,Saccade Peak Velocity,Saccade Peak Acceleration,Saccade Peak Deceleration,Saccade Direction
0,2,P01,0,OTHER,Default,Excellent,128.9448,img,1,141.658,0.5989,0.5257,0.4298,0.5191,0,0,960.5,494.5,0,0,0,1,0,0,1,0,0,0,0,0,960.5,494.5,567.9688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,P01,0,OTHER,Default,Excellent,137.2724,img,1,149.984,0.5991,0.5255,0.4300,0.5190,0,0,952.0,509.5,0,0,0,1,0,0,1,0,0,0,0,0,952.0,509.5,567.9048,27.4655,1.0,NaN,947.656,491.5955,133.1086,358.082,224.9734,0.3205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,P01,0,OTHER,Default,Excellent,145.6203,img,1,158.333,0.5990,0.5253,NaN,NaN,0,4,938.0,500.0,0,0,0,1,0,0,1,0,0,0,0,0,938.0,500.0,566.2654,13.3622,1.0,NaN,947.656,491.5955,133.1086,358.082,224.9734,0.3205,2.6143,-1689.4409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,P01,0,OTHER,Default,Excellent,153.9470,img,1,166.660,0.5989,0.5252,0.4299,0.5187,0,0,948.0,499.0,0,0,0,1,0,0,1,0,0,0,0,0,948.0,499.0,567.7859,41.4435,1.0,NaN,947.656,491.5955,133.1086,358.082,224.9734,0.3205,NaN,3372.4442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7,P01,0,OTHER,Default,Excellent,162.2703,img,1,174.983,0.5991,0.5251,0.4301,0.5187,0,0,969.5,515.0,0,0,0,1,0,0,1,0,0,0,0,0,969.5,515.0,567.7856,36.3849,1.0,NaN,947.656,491.5955,133.1086,358.082,224.9734,0.3205,NaN,-607.7654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Drop rows with GazeX or GazeY equals to NaN. This because these gazes are not mapped to any AOI
data = data.dropna(subset=['GazeX','GazeY'])

# Drop rows with both FixID and  SacID equal to NaN
data = data.dropna(subset=['FixID','SacID'], how='all')

# Keep only relevant colomns
fixationAndSaccadesDataWithAOIs = data[['Respondent','Timestamp','FixID', 'Fixation X', 'Fixation Y',
       'Fixation Start', 'Fixation End', 'Fixation Duration',
       'Fixation Dispersion', 'SacID', 'Saccade Start',
       'Saccade End', 'Saccade Duration', 'Saccade Amplitude',
       'Saccade Peak Velocity', 'Saccade Peak Acceleration',
       'Saccade Peak Deceleration', 'Saccade Direction']+AOIS].copy(deep=True)

# Map fixations and saccades to AOIs
"""
Goal: 
- Map fixations and saccades to AOIs
Context: 
- A fixation or saccade comprises several gazes.
- Following the NoteBook "1. Gaze Projections and AOIs Mapping", each gaze point is mapped to an AOI.
Problem: 
- Gazes belonging to the same fixation or saccade might not necessarily be mapped to the same AOI.
Solution: 
- When mapping fixations or saccades to AOIs, choose the AOI which is mapped to the majority of the gazes composing the fixation or the saccade
"""                                
fixationAndSaccadesDataWithAOIs = fixationAndSaccadesDataWithAOIs.groupby(['Respondent','FixID', 'Fixation X', 'Fixation Y',
       'Fixation Start', 'Fixation End', 'Fixation Duration',
       'Fixation Dispersion', 'SacID', 'Saccade Start',
       'Saccade End', 'Saccade Duration', 'Saccade Amplitude',
       'Saccade Peak Velocity', 'Saccade Peak Acceleration',
       'Saccade Peak Deceleration', 'Saccade Direction'],dropna=False).agg({aoi: (lambda x: pd.Series.mode(x)[0]) for aoi in AOIS} | {'Timestamp': lambda x: x.iloc[0]}) 


# Flatten fixationAndSaccadesDataWithAOIs (i.e., a hierarchically indexed pandas.DataFrame)
fixationAndSaccadesDataWithAOIs.columns= [x for x in list(fixationAndSaccadesDataWithAOIs.columns.get_level_values(0))]
fixationAndSaccadesDataWithAOIs = fixationAndSaccadesDataWithAOIs.reset_index()

# Sort fixationAndSaccadesDataWithAOIs by Respondent and Timestamp
fixationAndSaccadesDataWithAOIs = fixationAndSaccadesDataWithAOIs.sort_values(by=['Respondent','Timestamp'])

# Reset index
fixationAndSaccadesDataWithAOIs = fixationAndSaccadesDataWithAOIs.reset_index(drop=True)

In [8]:
# Preview fixation data
fixationAndSaccadesDataWithAOIs.head()

,Respondent,FixID,Fixation X,Fixation Y,Fixation Start,Fixation End,Fixation Duration,Fixation Dispersion,SacID,Saccade Start,Saccade End,Saccade Duration,Saccade Amplitude,Saccade Peak Velocity,Saccade Peak Acceleration,Saccade Peak Deceleration,Saccade Direction,Paragraph1,Paragraph2,Paragraph3,Paragraph4,Paragraph5,Paragraph6,Text_Area,SubFigure1,SubFigure2,SubFigure3,Figure,Formula,Timestamp
0,P01,1.0,947.6560,491.5955,133.1086,358.0820,224.9734,0.3205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0,0,1,0,0,0,0,0,137.2724
1,P01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,358.0820,433.0785,74.9965,5.6844,148.4457,9324.2426,-9634.6219,203.9321,0,0,0,1,0,0,1,0,0,0,0,0,362.2460
2,P01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,474.7339,491.4003,16.6664,0.6259,59.9309,4718.1428,-1362.1590,84.5597,0,0,0,0,0,0,1,0,0,0,0,0,478.8990
3,P01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,516.3954,566.4076,50.0122,6.6098,200.9994,8268.0361,-10127.0766,225.0000,0,0,0,0,0,0,1,0,0,0,0,0,520.5610
4,P01,2.0,460.5417,172.5694,566.4076,716.4244,150.0168,0.1536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,1,0,0,0,0,0,570.5892


In [9]:
# Export fixation data with AOIs to csv
fixationAndSaccadesDataWithAOIs.to_csv("data/fixationAndSaccadesDataWithAOIs.csv",  index=False)